In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd

# Load your dataset
df = pd.read_csv('Your_CSV_file.csv')

# Convert 'rating' and 'number_of_reviews' columns to numeric, invalid parsing will be set as NaN
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df['number_of_reviews'] = pd.to_numeric(df['number_of_reviews'].str.extract(r'(\d+)')[0], errors='coerce')

# Initialize the Dash app
rest_dash = dash.Dash(__name__)

# Layout of the app with filters for rating, food type, price range (coupon), and other columns
rest_dash.layout = html.Div([
    html.H1("Restaurant Dashboard"),

    # Label for rating slider
    html.Label("Select Rating"),
    
    # Slider for filtering by rating
    dcc.Slider(
        id='rating-slider',
        min=0,
        max=5,
        step=0.1,
        value=4,  # Default rating filter value
        marks={i: f'{i}' for i in range(6)},
        tooltip={"placement": "bottom", "always_visible": True}
    ),
    
    # Label for range of reviews slider
    html.Label("Select Range of Reviews"),
    
    # Range slider for filtering by number of reviews
    dcc.RangeSlider(
        id='reviews-slider',
        min=0,
        max=df['number_of_reviews'].max(),
        step=100,  # Step size
        value=[2000, 4000],  # Default range
        marks={i: f'{int(i)}' for i in range(0, int(df['number_of_reviews'].max()), 1000)},
        tooltip={"placement": "bottom", "always_visible": True}
    ),
    
    # Dropdown filter for selecting food type
    dcc.Dropdown(
        id='food-type-dropdown',
        options=[{'label': food, 'value': food} for food in df['food_type'].unique()],
        value=None,
        placeholder="Select food type"
    ),
    
    # Dropdown filter for price range
    dcc.Dropdown(
        id='price-range-dropdown',
        options=[{'label': price, 'value': price} for price in df['coupon'].unique()],
        value=None,
        placeholder="Select price range"
    ),
    
    # Dropdown filter for food score
    dcc.Dropdown(
        id='food-dropdown',
        options=[{'label': f'At least {i}', 'value': i} for i in sorted(df['food'].dropna().unique())],
        value=None,
        placeholder="Select minimum food score"
    ),
    
    # Dropdown filter for service score
    dcc.Dropdown(
        id='service-dropdown',
        options=[{'label': f'At least {i}', 'value': i} for i in sorted(df['service'].dropna().unique())],
        value=None,
        placeholder="Select minimum service score"
    ),
    
    # Dropdown filter for ambience score
    dcc.Dropdown(
        id='ambience-dropdown',
        options=[{'label': f'At least {i}', 'value': i} for i in sorted(df['ambience'].dropna().unique())],
        value=None,
        placeholder="Select minimum ambience score"
    ),
    
    # Div where filtered restaurant names will be displayed
    html.Div(id='restaurant-names')
])

# Callback to update restaurant names based on selected filters
@rest_dash.callback(
    Output('restaurant-names', 'children'),
    [
        Input('rating-slider', 'value'),
        Input('reviews-slider', 'value'),
        Input('food-type-dropdown', 'value'),
        Input('price-range-dropdown', 'value'),
        Input('food-dropdown', 'value'),
        Input('service-dropdown', 'value'),
        Input('ambience-dropdown', 'value')
    ]
)
def update_restaurant_names(selected_rating, selected_reviews_range, selected_food, selected_price, selected_food_score, selected_service_score, selected_ambience_score):
    # Copy the original dataframe to filter
    filtered_df = df.copy()

    # Apply filters based on inputs
    if selected_rating is not None:
        filtered_df = filtered_df[filtered_df['rating'] >= selected_rating]
    
    if selected_reviews_range is not None:
        filtered_df = filtered_df[
            (filtered_df['number_of_reviews'] >= selected_reviews_range[0]) &
            (filtered_df['number_of_reviews'] <= selected_reviews_range[1])
        ]
    
    if selected_food is not None:
        filtered_df = filtered_df[filtered_df['food_type'].str.strip() == selected_food.strip()]
    
    if selected_price is not None:
        filtered_df = filtered_df[filtered_df['coupon'].str.strip() == selected_price.strip()]
    
    if selected_food_score is not None:
        filtered_df = filtered_df[filtered_df['food'] >= selected_food_score]
    
    if selected_service_score is not None:
        filtered_df = filtered_df[filtered_df['service'] >= selected_service_score]
    
    if selected_ambience_score is not None:
        filtered_df = filtered_df[filtered_df['ambience'] >= selected_ambience_score]

    # If no restaurants match the criteria, show a message
    if filtered_df.empty:
        return "No restaurants found with the selected criteria."
    
    # Return only the first 10 filtered restaurant names, each on a new line
    top_10_restaurants = filtered_df['rest_name'].head(10).tolist()
    return html.Ul([html.Li(restaurant) for restaurant in top_10_restaurants])

# Run the app
if __name__ == '__main__':
    rest_dash.run_server(debug=True)